In [ ]:
from tkinter import *
from PIL import ImageTk, Image,ImageFilter
from tkinter import filedialog
import os
from tkinter.filedialog import askdirectory
import numpy
from imshowpair import *
import matplotlib.pyplot as plt
from skimage.util import random_noise
from math import log10, sqrt
import math

# <font color="black"><u>Exercice 1</u> :</font>

In [ ]:
#Q1 : Load an image and take its information

def open_file ():
    window = Tk()
    window.withdraw()
    file = filedialog.askopenfilename()
    return file

def ouvre():
    image=Image.open(open_file())
    return image

def image_param(image):
    return image.format,image.size,image.mode,image.palette

In [ ]:
image=ouvre()
print(image_param(image))

In [ ]:
#Q_2 : Save the image

def save_image(img,image,path,op):
    filename=os.path.splitext(path)[0]+op+f".{image_param(image)[0]}"
    img.save(filename)

In [ ]:
#Q3 : Resize the image

#There is a predefinend fonction to resize:iamge.resize((round(image.size[0]*0.5), round(image.size[1]*0.5)))

def redimenssion(x,y):
    file=open_file()
    image=Image.open(file)
    image.thumbnail((x,y))
    save_image(image,image,file,"_thumbnail")
    image.show() 

In [ ]:
redimenssion(700,200)

In [ ]:
def #Q4 : Convert the color system of the image

def color_system():
    color=["RGB","L","1","CMYK"]
    file=open_file()
    image=Image.open(file)
    org=image_param(image)[2]
    for i in color:
        if i!=org:
            name=f"_{i}"
            img=image.convert(i)
            save_image(img,image,file,f"_{i}")

In [ ]:
color_system()

In [ ]:
#Q5 : Geometry modifications

def geometry(size,retation):
    file=open_file()
    image=Image.open(file)
    img1=image.resize(size)
    img1.rotate(retation).show()

In [ ]:
geometry((200,200),90)

In [ ]:
#Q6 : Separate the different channels of the image

def separe():
    file=open_file()
    image=Image.open(file)
    return image.split()

def separe2(n):                        # n est la valeur de plan (0,1,2) 
    file=open_file()
    image=Image.open(file)
    a=numpy.array(image)
    if n==0:
        i=1;j=2
    elif n==1:
        i=0;j=2
    elif n==2:
        i=1;j=0
    a[:,:,i] *=0
    a[:,:,j] *=0
    a = Image.fromarray(a)
    a.show()

In [ ]:
separe2(1)

In [ ]:
#Q7 : Trimming the image

def decoper(x1,y1,x2,y2):
    file=open_file()
    image=Image.open(file)
    cord=(x1,y1,x2,y2)
    img=image.crop(cord)
    img.show()

In [ ]:
decoper(20,20,100,100)

In [ ]:
#Q9 : Convert image to line list

def liste():
    image=ouvre()
    return numpy.array(image)

In [ ]:
#Q10 : Make a transition between 2 images

def new_image(t):
    im1=ouvre()
    im2=ouvre()
    im2=im2.resize(im1.size)
    A=numpy.array(im1)
    B=numpy.array(im2)
    C=(1-t)*A+t*B
    return Image.fromarray((C).astype(numpy.uint8))

In [ ]:
#Fonction predefine for merge two images:
C=Image.blend(A,B,0.5)
C.show()

In [ ]:
new=new_image(0.5)
new.show()

In [ ]:
#Q11 : Display 2 images side by side

def show_pair(im1,im2):
    plt.figure(figsize=(100,100))
    imshowpair(im1,im2,cmap='gray')

In [ ]:
im1=ouvre()
im2=ouvre()
show_pair(im1,im2)

# <font color="black"><u>Exercice 2</u> :</font>

In [ ]:
#Q1 : Noises an image with a fixed type of noise and display side by side, the original image and the noisy ones

def bruit(image,typ):
    noise_img = random_noise(image, mode=typ)
    noise_img = np.array(255*noise_img, dtype = 'uint8')
    noise_img= Image.fromarray(noise_img)
    return noise_img

In [ ]:
im=liste()
im2=bruit(im,'gaussian')
show_pair(im,im2)

In [ ]:
#Q3 : Create an averaging filter, a Gaussian filter and a median filter

def moyenneur(n):
    moy=numpy.ones((n,n))/(n**2)
    return moy
    
def gaussian(sigma,n):
    g = numpy.zeros((n,n))
    ind=(n-1)//2
    som = 0
    for i in range(-ind,ind+1):
        for j in range(-ind,ind+1):
            g[j+ind][i+ind] = (1/(2*math.pi*(sigma*sigma)))*(math.exp(-((i*i+j*j)/(2*(sigma*sigma)))))
    return g
    
def median(matrice,n):
    ind=(n-1)//2
    vect=sorted(matrice.ravel().tolist())
    return vect[ind]

In [ ]:
#Q4 : Filter a noisy image by applying the following filters: Averaging filter, Gaussian size filter and Median filter

#Fonctio predefine for make image blur: 
#blurred_image = im.filter(ImageFilter.GaussianBlur(radius))
#blurred_image = im.filter(ImageFilter.MedianFilter(size))

def convolution_gris(img,fltr):                    # img sous forme array 
    px = int( (len(fltr)-1) / 2 )
    im = img.copy()
    imax = img.shape[1] - px
    for i in range(px,imax):
        for j in range(px,img.shape[0]-px):
            im[j][i] = numpy.sum(img[j-px:j+px+1,i-px:i+px+1] * fltr)
    return im

def filtre_median_gris(img,n):                # img sous forme array
    px = int( (n-1) / 2 )
    im = img.copy()
    imax = img.shape[1] - px
    for i in range(px,imax):
        for j in range(px,img.shape[0]-px):
            im[j][i] =median(img[j-px:j+px+1,i-px:i+px+1],n)
    return im

def convolution_rgb(img,fltr):               # img sous forme array
    im=img.copy()
    for i in range(3):
        im[:,:,i]=convolution_gris(img[:,:,i],fltr)
    return Image.fromarray(im)

def filtre_median_rgb(img,n):
    im=img.copy()
    for i in range(3):
        im[:,:,i]=filtre_median_gris_gris(img[:,:,i],fltr)
    return Image.fromarray(im)

In [ ]:
moy=moyenneur(5)
convolution_gris(numpy.array(im2),moy)

In [ ]:
moy=gaussian(1,7)
convolution_gris(numpy.array(im2),moy)

In [ ]:
filtre_median_gris(numpy.array(im2),3)

In [ ]:
#Q5 : Evaluate an objective measure called Peak Signal to Noise Ratio (PSNR)

def pnsr(orig,fltr):
    mse = numpy.mean((orig.astype(numpy.int8) - fltr.astype(numpy.int8)) ** 2)
    if(mse == 0):
        return 100
    max_pixel = 255.0
    psnr = 10 * log10(max_pixel / sqrt(mse))
    return psnr

In [ ]:
orig=liste()
p=pnsr(orig,convolution_gris(numpy.array(im2),moy))
print(p)

In [ ]:
#Q6 : Compute the PSNR for each simulation previously performed

def compare(p):
    if p>=20 and p<=40 :
        print("Filtrage est de bonne qualite ")
    else :
        print("Filtrage est de mauvaise qualite ")

In [ ]:
compare(p)

# <font color="black"><u>Exercice 3</u> :</font>

In [ ]:
#Q1 : Calculates the histogram of an image

def histo(arr):
    list_arr=arr.ravel().tolist()
    list_deg=[];list_val=[]
    for i in range(256):
        list_deg.append(i)
        list_val.append(list_arr.count(i))
    return list_deg,list_val

def hito_rgb(arr):
    r,g,b=separ()
    r=histo(r)[1]
    g=histo(g)[1]
    b=histo(b)[1]
    return r,g,b

In [ ]:
im1=liste()
h=histo(im1)[1]
plt.hist(h,width=5,bins=range(255))

In [ ]:
#Q2 : Evaluates the cumulative histogram

def histo_cum(arr):
    h=[]
    hi=histo(arr)[1]
    for i in range(len(hi)):
        h.append((1/(arr.shape[0]*arr.shape[1]))*(sum(hi[0:i+1])))
    return h

def histo_cum_rgp(arr):
    r,g,b=separ()
    r=histo_cum(r)
    g=histo_cum(g)
    b=histo_cum(b)
    return r,g,b

In [ ]:
hc=histo_cum(im1)
print(hc)

In [ ]:
#Q3_A : Extend the dynamics of an image

def dynamic(arr):
    Inew=arr.copy()
    Inew=(arr-arr.min())*(255/(arr.max()-arr.min()))
    return Inew

In [ ]:
im=Image.fromarray(dynamic(im1))
im.show()

In [ ]:
#Enhance the contrast of an image

def contrast(arr):
    list_arr=arr.ravel().tolist()
    con=((max(list_arr)-min(list_arr))/(max(list_arr)+min(list_arr)))
    return con

In [ ]:
contrast(im1)

In [ ]:
#Q3_B : Enhance the egalisation of an image

def egalistation(arr,d):
    Inew=arr.copy()
    liste=histo_cum(arr)
    for i in range(arr.shape[1]):
        for j in range(arr.shape[0]):
            Inew[j][i]=liste[arr[j][i]]*((2**d)-1)
    return Inew

In [ ]:
im2=egalistation(im1,8)
im3=Image.fromarray(im2)
show_pair(Image.fromarray(im1),im3)

In [ ]:
#Q5 : Evaluate the quality of a processed image by calculating :
        #Root Mean Square Error MSE
        #Signal to Noise Ratio SNR
        #Peak Signal to Noise Ratio PSNR

def qualite(orig,fltr):
    list_arr=orig.ravel().tolist()
    v=(1/(orig.shape[0]*orig.shape[1])*(sum(list_arr)**2))
    mse = numpy.mean((orig.astype(numpy.int8) - fltr.astype(numpy.int8)) ** 2)
    snr=10*math.log10(v/mse)
    print("mse = ",mse,'\n',"snr = ",snr,'\n',"pnsr = ",pnsr(orig,fltr))

In [ ]:
qualite(im1,im2)